In [1]:
import math
from collections import defaultdict

In [2]:
bi_gram = open('count_2w.txt').read().splitlines()
bi_gram = [x.split('\t') for x in bi_gram]

In [3]:
# transform ngram to rank
bi_words = [[x,int(y)] for x,y in bi_gram]
bi_words = sorted(bi_words, key=lambda x: x[1], reverse=True)

words_rank = []
count = 1
for w,c in bi_words:
    words_rank.append([count, w, c])
    count += 1

In [4]:
words_rank

[[1, 'of the', 2766332391],
 [2, 'in the', 1628795324],
 [3, 'to the', 1139248999],
 [4, 'on the', 800328815],
 [5, 'for the', 692874802],
 [6, 'and the', 629726893],
 [7, 'to be', 505148997],
 [8, 'is a', 476718990],
 [9, 'with the', 461331348],
 [10, 'from the', 428303219],
 [11, 'by the', 417106045],
 [12, 'at the', 416201497],
 [13, 'of a', 387060526],
 [14, 'in a', 364730082],
 [15, 'will be', 356175009],
 [16, 'that the', 333393891],
 [17, 'do not', 326267941],
 [18, 'is the', 306482559],
 [19, 'to a', 279146624],
 [20, 'is not', 276753375],
 [21, 'for a', 274112498],
 [22, 'with a', 271525283],
 [23, 'as a', 270401798],
 [24, '<S> and', 261891475],
 [25, 'of this', 258707741],
 [26, '<S> the', 258483382],
 [27, 'it is', 245002494],
 [28, 'can be', 230215143],
 [29, 'If you', 210252670],
 [30, 'has been', 196769958],
 [31, 'the same', 186235801],
 [32, 'does not', 180844574],
 [33, 'can not', 180466484],
 [34, 'and a', 178504444],
 [35, 'in this', 174166565],
 [36, 'one of', 1738

In [5]:
def words_pre(word):
    words_list = []
    for r,w,c in words_rank:
        words = w.split(' ')
        if words[0] == word:
            words_list.append((r,w,c))
            
    return words_list

In [6]:
def words_post(word):
    words_list = []
    for r,w,c in words_rank:
        words = w.split(' ')
        if words[1] == word:
            words_list.append((r,w,c))
            
    return words_list

## Mutual Information

In [7]:
total_count = sum([c for r,w,c in words_rank])

def calculate_mutual_info(target_words):
    pre_word, post_word = target_words.split(' ')
    
    pre_rank = words_pre(pre_word)
    post_rank = words_post(post_word)
    total_pre = sum([c for r,w,c in pre_rank])
    total_post = sum([c for r,w,c in post_rank])
    
    prob_pre = total_pre / total_count
    prob_post = total_post / total_count

    target_count = [c for r,w,c in words_rank if w==target_words][0]
    prob_target = target_count / total_count

    mutual_info = prob_target / (prob_pre * prob_post)
    
    return mutual_info

In [8]:
calculate_mutual_info('hot dog')

119.69350844033332

In [ ]:
mutual_info_dict = defaultdict(lambda x: 0.0)

for r,w,c in words_rank:
    mutual_info = calculate_mutual_info(w)
    mutual_info_dict[w] = mutual_info
    
mutual_info_dict = {k:v for k,v in sorted(mutual_info_dict.items(), key=lambda x: x[1])}

In [ ]:
mutual_info_dict

## Rank Ratio

In [9]:
def calculate_rank_ratio(target_words):
    pre_word, post_word = target_words.split(' ')
    pre = words_pre(pre_word)
    post = words_post(post_word)

    # calculate expected rank
    pre_avg_rank = int(sum([r for r,w,c in pre]) / len(pre))
    post_avg_rank = int(sum([r for r,w,c in post]) / len(post))
    expected_rank = int(math.sqrt(pre_avg_rank * post_avg_rank))
    
    # calculate actual rank
    target_rank = [r for r,w,c in words_rank if w==target_words][0]
    
    # calculate rank ratio
    rank_ratio = expected_rank / target_rank
    
    return rank_ratio

In [10]:
calculate_rank_ratio('hot dog')

2.2025535182874147

In [ ]:
rank_ratio_dict = defaultdict(lambda x: 0.0)

for r,w,c in words_rank:
    rank_ratio = calculate_rank_ratio(w)
    rank_ratio_dict[w] = rank_ratio
    
rank_ratio_dict = {k:v for k,v in sorted(rank_ratio_dict.items(), key=lambda x: x[1])}

In [ ]:
rank_ratio_dict